In [1]:
import os 
import zipfile as zf
from cryptography.fernet import Fernet as fn
import random
import string
import json
import time
import copy
import shutil

In [2]:
error = 0

In [3]:
def versionControl(version = "0.0.2", source="Restore Backup", software="Project Ultra Backup", author="Anjal.P"):
    ver = version
    sour = source
    software = software
    creat = author
    return ver, sour, software, creat

In [4]:
def errorSave(where_to_save, error_report, root_directory):
    global error
    where_to_save = where_to_save + "//"
    print("look at: ", where_to_save + "Error Report UB.txt")
    ver_control, software_name, creator, section = versionControl()
    date_error = str(time.localtime(time.time()).tm_mday) + "." + str(time.localtime(time.time()).tm_mon) + "." + str(time.localtime(time.time()).tm_year)
    if os.path.exists(where_to_save + "Error Report UB.txt")==False:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Restore: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except Exception as e:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))
    else:
        try:
            with open(where_to_save + "Error Report UB.txt", 'a') as error_file:
                error_file.write("--"*50 + "\n")
                error_file.write("Errors :" + "[" +str(error) + "]" + "\n")
                error_file.write("On Running Backup: " + root_directory + "\n")
                error_file.write("Created On: " + date_error + "\n")
                error_file.write("Report: \n")
                for eachError in error_report:
                    error_file.write("     > " + eachError + " " + error_report[eachError])
                error_file.write("This is an automated report generated by " + str(software_name) + " " + str(ver_control) + " " + "Author: " + str(creator) + "\n")
                error_file.write("--"*50 + "\n")
                error_file.close()
        except:
            print("Error even creating the error log .txt at: " + where_to_save + "Error Report UB.txt")
            print("Error Report: " + str(error_report))
            print("Error on errorSave: " + str(e))

In [5]:
def scanBackupDirectory(backupDirectory, ubmapDirectory, toRecover, workingDirectory):
    backupScan = {"UB_date":{}, ".UBmap":{}, ".val":{}}    #    in arangement as : {"UB_date": [], ".UBmap": [], ".val": []} of selected folder toRecover.
    count = 0
    for folder, subfolder, files in os.walk(backupDirectory):   #scanning the backup directory of the restor of the toRecover folder backups
        if "oldBackup" not in folder:
            if toRecover==os.path.basename(folder):
                for file in files:
                    if file[-4:]==".val":    #scanned .val and folder name.
                        backupScan[".val"][count] = folder.replace("\\", "//") + "//" + file
                        backupScan["UB_date"][count] = folder.replace("\\", "//")
                        count += 1
    for folder, subfolder, files in os.walk(ubmapDirectory):    #reading the ubmap file and arranging accroding to the orede of .val files.
        if "oldUBmap" not in folder:
            for valfiles in backupScan[".val"]:
                for ubfiles in files:
                    if os.path.basename(backupScan[".val"][valfiles])[:-3]==ubfiles[:-5]:
                        backupScan[".UBmap"][valfiles] = folder.replace("\\", "//") + "//" + ubfiles
    if len(backupScan[".val"])==0:
        print("Sorry... no backup to restore on: " + toRecover)
    if len(backupScan[".val"])!=len(backupScan[".UBmap"]):
        print("It seems that some UBmap files are missing, taking measures for the optimal backups")
    return backupScan

In [6]:
def scanDate(directoryToScan):
    scan = {}
    for folder, subfolder, files in os.walk(directoryToScan):
        for file in files:
            if file[-4:]==".eUB" or file[-4:]==".sUB":
                scan[file] = folder.replace("\\", "//")
    return scan

In [7]:
def restore(restoreDirectory, backupDirectory, toRecover, workingDirectory, backupScan):
    global error
    errorReport = {}
    if os.path.isdir(restoreDirectory)==False:
        try:
            os.makedirs(restoreDirectory)
        except Exception as e:
            error += 1
            errorReport["er_create_restoredir"] = restoreDirectory
            errorReport["Error: "] = str(e)
            errorReport["Plan of action: "] = "Please select another directory."
            errorSave(workingDirectory, errorReport, restoreDirectory)
            return
    for eachUBFile in backupScan[".UBmap"]:
        try: 
            with open(backupScan[".UBmap"][eachUBFile], 'r') as ubfile:    #readingt he ubmap file and saving it to a variable ubstore
                try:
                    ubstore = json.load(ubfile)
                    ubfile.close()
                except Exception as e:
                    ubfile.close()
                    error += 1
                    errorReport["er_load_ubmap"] = backupScan[".UBmap"][eachUBFile]
                    errorReport["Error: "] = str(e)
                    errorSave(workingDirectory, errorReport, ubmapDirectory)
        except Exception as e:
            error += 1
            errorReport["er_open_UBmap"] = backupScan[".UBmap"][eachUBFile]
            errorReport["Error: "] = str(e)
            errorReport["Effect: "] = "Some files cannot be backed up due to, not accessible .UBmap or try again"
            errorSave(workingDirectory, errorReport, ubmapDirectory)
        scanUBFiles = scanDate(backupScan["UB_date"][eachUBFile])    #scan the eUB and sUB files in the folder.
        for eachData in ubstore:             #restoring the files
            if os.path.isdir(restoreDirectory + eachData)==False:
                try: 
                    os.makedirs(restoreDirectory + eachData)          #creating the folder.
                except Exception as e:
                    error += 1
                    errorReport["er_create_dir"] = restoreDirectory + eachData
                    errorReport["Error: "] = str(e)
                    errorReport["Plan of action: "] = "Please select another directory."
                    errorSave(workingDirectory, errorReport, restoreDirectory)
                    return
            for withinData in ubstore[eachData]["z_n"]:
                if type(ubstore[eachData]["z_n"][withinData])==type({}):
                    badFile = 0
                    randName = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
                    for eachSplit in ubstore[eachData]["z_n"][withinData]:
                        if ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB" in scanUBFiles:
                            try: 
                                with open(workingDirectory + randName + ".zip", 'ab') as sUBZipFile:
                                    try: 
                                        with open(scanUBFiles[ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"] + "//" + ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB", 'rb') as sUBFile:
                                            try: 
                                                sUBData = sUBFile.read()
                                                sUBFile.close()
                                            except Exception as e:
                                                sUBFile.close()
                                                error += 1
                                                errorReport["er_read_.sUB"] = scanUBFiles[ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"] + "//" + ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"
                                                errorReport["Error: "] = str(e)
                                                errorReport["Plan of action: "] = "Moving to next file to restore."
                                                errorSave(workingDirectory, errorReport, backupDirectory)
                                        decryptKey = (ubstore[eachData]["k"][withinData]).encode()
                                        decryptKey = fn(decryptKey)
                                        try:
                                            decryptFile = decryptKey.decrypt(sUBData)
                                        except Exception as e:
                                            error += 1
                                            errorReport["er_decrypt_file"] = scanUBFiles[ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"] + "//" + ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"
                                            errorReport["Error: "] = str(e)
                                            errorReport["Planof action: "] = "Looks that the key is not suitable, or the UBmap file is altered...."
                                            errorSave(workingDirectory, errorReport, backupDirectory)
                                        try: 
                                            sUBZipFile.write(decryptFile)
                                            sUBZipFile.close()
                                        except Exception as e:
                                            sUBZipFile.close()
                                            error += 1
                                            errorReport["er_writing_zip"] = workingDirectory + randName + ".zip"
                                            errorReport["Error: "] = str(e)
                                            errorReport["Plan of action: "] = "Moving to next file."
                                            errorSave(workingDirectory, errorReport, workingDirectory)
                                    except Exception as e:
                                        sUBZipFile.close()
                                        error += 1
                                        errorReport["er_open_.sUB"] = scanUBFiles[ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"] + "//" + ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB"
                                        errorReport["Error: "] = str(e)
                                        errorReport["Plan of action: "] = "Moving to next file"
                                        errorSave(workingDirectory, errorReport, backupDirectory)
                            except Exception as e:
                                error += 1
                                errorReport["er_open/create_zip"] = workingDirectory + randName + ".zip"
                                errorReport["Error: "] = str(e)
                                errorReport["Plan of action: "] = "Continue with the next files....."
                                errorSave(workingDirectory, errorReport, workingDirectory)
                        else:
                            badFile = 1
                            print("Some files are missing from the backup directory.")
                            print("Restore of: " + ubstore[eachData]["z_n"][withinData] + " is currently not possible...")
                            print("Missing file: " + ubstore[eachData]["z_n"][withinData][eachSplit] + ".sUB")
                    if badFile!=1:
                        try:
                            openZip = zf.ZipFile(workingDirectory + randName + ".zip")
                            try: 
                                openZip.extractall(restoreDirectory + eachData + "//")
                                openZip.close()
                            except Exception as e:
                                openZip.close()
                                error += 1
                                errorReport["er_extracting_files: "] = workingDirectory + randName + ".zip"
                                errorReport["Error: "] = str(e)
                                errorReport["Plan of action: "] = "Move to next file" + " Cause amy be due to corrupted zip."
                                errorSave(workingDirectory, errorReport, workingDirectory)
                        except Exception as e:
                            error += 1
                            errorReport["er_open_zip"] = workingDirectory + randName + ".zip"
                            errorReport["Error: "] = str(e)
                            errorReport["Plan of action: "] = "Moving to he next file"
                            errorSave(workingDirectory, errorReport, workingDirectory)
                        try: 
                            os.remove(workingDirectory + randName + ".zip")
                        except Exception as e:
                            error += 1
                            errorReport["er_remove_zip"] = workingDirectory + randName + ".zip"
                            errorReport["Error: "] = str(e)
                            errorSave(workingDirectory, errorReport, workingDirectory)
                else:
                    if ubstore[eachData]["z_n"][withinData] + ".eUB" in scanUBFiles:
                        try: 
                            with open(scanUBFiles[ubstore[eachData]["z_n"][withinData] + ".eUB"] + "//" + ubstore[eachData]["z_n"][withinData] + ".eUB", 'rb') as eUBFile:
                                try: 
                                    eUBData = eUBFile.read()
                                    eUBFile.close()
                                except Exception as e:
                                    eUBFile.close()
                                    error += 1
                                    errorReport["er_read_.eUB"] = scanUBFiles[ubstore[eachData]["z_n"][withinData] + ".eUB"] + "//" + ubstore[eachData]["z_n"][withinData] + ".eUB"
                                    errorReport["Error: "] = str(e)
                                    errorReport["Plan of action: "] = "Moving to next file"
                                    errorSave(workingDirectory, errorReport, workingDirectory)
                        except Exception as e:
                            error += 1
                            errorReport["er_open_.eUB"] = scanUBFiles[ubstore[eachData]["z_n"][withinData] + ".eUB"] + "//" + ubstore[eachData]["z_n"][withinData] + ".eUB"
                            errorReport["Error: "] = str(e)
                            errorReport["Plan of action: "] = "Moving to next file"
                            errorSave(workingDirectory, errorReport, workingDirectory)
                        decryptKey = (ubstore[eachData]["k"][withinData]).encode()
                        decryptKey = fn(decryptKey)
                        try:
                            decryptFile = decryptKey.decrypt(eUBData)
                        except Exception as e:
                            error += 1
                            errorReport["er_decrypt_file"] = scanUBFiles[ubstore[eachData]["z_n"][withinData] + ".eUB"] + "//" + ubstore[eachData]["z_n"][withinData] + ".eUB"
                            errorReport["Error: "] = str(e)
                            errorReport["Planof action: "] = "Looks that the key is not suitable, or the UBmap file is altered...."
                            errorSave(workingDirectory, errorReport, backupDirectory)
                        try: 
                            with open(workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip", 'wb') as zipFile:
                                try: 
                                    zipFile.write(decryptFile)
                                    zipFile.close()
                                except Exception as e:
                                    zipFile.close()
                                    error += 1
                                    errorReport["er_writing_zip"] = workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip"
                                    errorReport["Error: "] = str(e)
                                    errorReport["Paln of action: "] = "Moving to next files."
                                    errorSave(workingDirectory, errorReport, workingDirectory)
                        except Exception as e:
                            error += 1
                            errorReport["er_create_zip"] = workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip"
                            errorReport["Error: "] = str(e)
                            errorReport["Paln of action: "] = "Moving to next files."
                            errorSave(workingDirectory, errorReport, workingDirectory)
                        try: 
                            openZip = zf.ZipFile(workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip")
                            openZip.extractall(restoreDirectory + eachData + "//")
                            openZip.close()
                        except Exception as e:
                            error += 1
                            errorReport["er_open_zip"] = workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip"
                            errorReport["Error: "] =str(e)
                            errorReport["Plan of action: "] = "Not able to open zip due to corruption. Skipping the file."
                            errorSave(workingDirectory, errorReport, workingDirectory)
                        try: 
                            os.remove(workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip")
                        except Exception as e:
                            error += 1
                            errorReport["er_remove_zip"] = workingDirectory + ubstore[eachData]["z_n"][withinData] + ".zip"
                            errorReport["Error: "] = str(e)
                            errorSave(workingDirectory, errorReport, workingDirectory)  

In [8]:
def restoreBackup(backupDirectory, restoreDirectory, ubmapDirectory, workingDirectory, toRecover):
    global error
    errorReport = {}
    if os.path.isdir(backupDirectory)==False:
        error += 1
        errorReport["dO not exist"] = "Backup directory: " + backupDirectory + " do not exist."
        errorSave(workingDirectory, errorReport, backupDirectory)
        return
    if os.path.isdir(restoreDirectory)==False:
        try:
            os.makedirs(restoreDirectory)
        except Exception as e:
            error += 1
            errorReport["er_crete_folder"] = restoreDirectory
            errorReport["Error: "] = str(e)
            errorReport["Plan of action: "] = "Choose a different folder."
            errorSave(workingDirectory, errorReport, restoreDirectory)
            return
    if os.path.isdir(ubmapDirectory)==False:
        error += 1
        errorReport["dO not exist"] = "Ubmap Directory: " + ubmapDirectory + " do not exist."
        errorSave(workingDirectory, errorReport, ubmapDirectory)
        return
    if os.path.isdir(workingDirectory)==False:
        try:
            os.makedirs(workingDirectory)
        except Exception as e:
            error += 1
            errorReport["er_crete_folder"] = workingDirectory
            errorReport["Error: "] = str(e)
            errorReport["Plan of action: "] = "Choose a different folder."
            errorSave(workingDirectory, errorReport, workingDirectory)
            return
    try:
        backupScan = scanBackupDirectory(backupDirectory + "//", ubmapDirectory + "//", toRecover, workingDirectory + "//")
    except Exception as e:
        error += 1
        errorReport["er_backupscan: "] = str(e)
        errorSave(workingDirectory, errorReport, backupDirectory)
        print("Something went wrong.")
        return
    try:
        restore(restoreDirectory + "//", backupDirectory + "//", toRecover, workingDirectory + "//", backupScan)
    except Exception as e:
        error += 1
        errorReport["er_restore: "] = str(e)
        errorSave(workingDirectory, errorReport, restoreDirectory)
        print("Something went wrong.")
        return
    return "Success"